In [1]:
# -*- coding: utf8 -*-
import pymongo
import json
import access_PlaceAPI_tool
import facebook
import io
import uniout
import requests
from pymongo import MongoClient
import re

In [2]:
#graph api
with io.open('config_secret_fb.json') as cred_fb:
    creds = json.load(cred_fb)
    graph = facebook.GraphAPI(access_token=creds['access_token'])

In [3]:
#pretty print
def pp(o):
    print json.dumps(o,indent=1,ensure_ascii=False)

# Get Fanpage Data

In [43]:
class fanpage:
    def __init__(self,fid):
        self.fid = fid
        self.obj = None
    def get_obj(self, refresh=False):
        if self.obj is None or refresh==True:
            obj = graph.get_object(self.fid,fields='fan_count,name,talking_about_count,posts.limit(5)')
            print 'fanpage '+self.fid+' send request to FB'
            self.obj = obj
        return self.obj
    def get_name(self):
        try:
            return self.get_obj()['name']
        except:
            print  self.fid+' has no \'name\' attribute'
    def get_fan_count(self):
        try:
            return self.get_obj()['fan_count']
        except:
            print self.fid+' has no \'fan_count\' attribute'
    def get_talking_about_count(self):
        try:
            return self.get_obj()['talking_about_count']
        except:
            print self.fid+' has no \'talking_about_count\' attribute'
    def get_latest_five_post(self):
        try:
            return [i['id'] for i in self.get_obj()['posts']['data']]
        except:
            print self.fid+' has no \'posts\'\'data\' attribute'

In [11]:
fp = fanpage('126949930763408')

In [12]:
print fp.fid
print fp.get_name()
print fp.get_fan_count()
print fp.get_talking_about_count()
print fp.get_latest_five_post()

126949930763408
fanpage 126949930763408 send request to FB
BMW Taiwan總代理 汎德
470962
14174
[u'126949930763408_669653613159701', u'126949930763408_666251120166617', u'126949930763408_664195573705505', u'126949930763408_666235246834871', u'126949930763408_663405040451225']


# Get Post Data

In [32]:
class post:
    def __init__(self,pid):
        self.pid = pid
        self.obj = None
    def get_obj(self, refresh=False):
        if self.obj is None or refresh==True:
            obj = graph.get_object(self.pid,fields='message,created_time,attachments{type},shares')
            print 'post '+self.pid+' send request to FB'
            self.obj = obj
        return self.obj
    def get_message(self):
        try:
            return self.get_obj()['message']
        except:
            print  self.pid+' has no \'message\' attribute'
    def get_created_time(self):
        try:
            return self.get_obj()['created_time']
        except:
            print self.pid+' has no \'created_time\' attribute'
    def get_post_type(self):
        try:
            return self.get_obj()['attachments']['data'][0]['type']
        except:
            print self.pid+' has no \'post_type\' attribute'
#     def get_share_count(self):
#         try:
#             return self.get_obj()['shares']['count']
#         except:
#             print self.pid+' has no \'share_count\' attribute'

In [33]:
post_test = post('127581957282188_1501151223258581')

In [34]:
print post_test.pid
print post_test.get_message()
print post_test.get_created_time()
print post_test.get_post_type()

127581957282188_1501151223258581
post 127581957282188_1501151223258581 send request to FB
換季時刻，肌膚最有感！
泛紅、乾燥、脫皮…各種狀況一次上門💢

安撫鬧情緒的肌膚
就要靠修護大師👉✨角鯊烯修護保濕面膜✨
保濕潤澤你的狀況肌，並開啟防禦機制

🌟立即【留言】大聲呼喊：
「換季狀況肌就靠__________修護大師來拯救！」
2017-02-17T12:00:02+0000
photo


# Get Post_Details Data

In [80]:
class post_detail:
    def __init__(self,pid):
        self.pid = pid
    def get_share_count(self):
        count = 0
        try:
            obj = graph.get_object(self.pid,fields='shares')
            print 'post_detail_share '+self.pid+' send request to FB'
            count = obj['shares']['count']
            return count
        except:
            print self.pid+' obj has no \'shares\' attribute'
            return count
    def get_like_count(self):
        count = 0
        next_index = ""
        while True:
            try:
                obj = graph.get_connections(self.pid, connection_name='likes',after=next_index)
                print 'post_detail_like '+self.pid+' send request to FB'
                count = count + len(obj['data'])
#                 print count
                if "next" in obj['paging']:
                    next_index = obj['paging']['cursors']['after']
#                     print next_index
                else:
                    break
            except:
                print self.pid+' cannot get \'like_count\' from Graph API'
                break
        return count
    def get_comment_count(self):
        count = 0
        next_index = ""
        while True:
            try:
                obj = graph.get_connections(self.pid, connection_name='comments',after=next_index)
                print 'post_detail_commet '+self.pid+' send request to FB'
                count = count + len(obj['data'])
#                 print count
                if "next" in obj['paging']:
                    next_index = obj['paging']['cursors']['after']
#                     print next_index
                else:
                    break
            except:
                print self.pid+' cannot get \'comment_count\' from Graph API'
                break
        return count
    def get_friend_tagging_count(self):
        count = 0
        next_index = ''
        while True:
            try:
                obj = graph.get_connections(self.pid, connection_name='comments',fields='message_tags',after=next_index)
                print 'post_detail_friend_tagg '+self.pid+' send request to FB'
                for i in obj['data']:
                    if 'message_tags' in i:
                        count = count+len(i['message_tags'])
                if "next" in obj['paging']:
                    next_index = obj['paging']['cursors']['after']
                else:
                    break
            except:
                print self.pid+' cannot get \'friend_tagging_count\' from Graph API'
                break
        return count

In [39]:
pda = post_detail('126949930763408_673905422734520')

In [40]:
print 'share_count: ',pda.get_share_count()
print 'like_count: ',pda.get_like_count()
print 'comment_count: ',pda.get_comment_count()
print 'friend_tagg_count: ',pda.get_friend_tagging_count()

share_count:  235
like_count:  5465
comment_count:  78
friend_tagg_count:  37


# Textmining for Classifying Posts

In [15]:
#po文是否有URL
def has_url(pid):
    obj = graph.get_object(pid, fiedls='message')
    if len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', obj['message'])) == 0:
        return False
    else:
        return True

In [16]:
has_url("126949930763408_654531844671878")

False

In [23]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2017-03-05 06:41:25'

# Insert Data into MongoDB

In [49]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [50]:
import datetime

In [51]:
#client.database_names()

In [52]:
db = client['fanpage_research']
coll_fanpage = db['fanpage']
coll_post = db['post']
coll_post_detail = db['post_detail']

In [45]:
with open('fid_list_2.json') as f:
    fid_list = json.load(f)

In [62]:
def update_fanpage_data(fid):
    fp = fanpage(fid)
    latest_five = fp.get_latest_five_post()
    fanpage_record = {
         "fid": fp.fid,
         "name": fp.get_name(),
         "fan_count": fp.get_fan_count(),
         "talking_about_count":fp.get_talking_about_count(),
         "date": datetime.datetime.utcnow()}
    print coll_fanpage.insert_one(fanpage_record)
    return latest_five

In [63]:
print update_fanpage_data('126949930763408')

fanpage 126949930763408 send request to FB
[u'126949930763408_674545299337199', u'126949930763408_673905422734520', u'126949930763408_669660796492316', u'126949930763408_671004546357941', u'126949930763408_669573229834406']


In [55]:
def update_post_data(fid,pid):
    result = coll_post.find_one({'pid':pid})
    if result is None:
        p = post(pid)
        post_record ={
            'fid':fid,
            'pid':p.pid,
            'message':p.get_message(),
            'post_type':p.get_post_type(),
            'created_time':p.get_created_time(),
            'date':datetime.datetime.utcnow()
                }
        print coll_post.insert_one(post_record) 

In [56]:
update_post_data('126949930763408','126949930763408_673905422734520')

post 126949930763408_673905422734520 send request to FB


In [57]:
def update_post_detail_data(pid):
    p_d = post_detail(pid)
    post_detail_record = {
        'pid':p_d.pid,
        'share_count': p_d.get_share_count(),
        'like_count':p_d.get_like_count(),
        'comment_count':p_d.get_comment_count(),
        'friend_tagg_count':p_d.get_friend_tagging_count(),
        'date': datetime.datetime.utcnow()}
    print coll_post_detail.insert_one(post_detail_record)

In [58]:
update_post_detail_data('126949930763408_673905422734520')

# Main Function (To run weekly)

In [67]:
def update_db_weekly(fid_list):
    for each_obj in fid_list:
        fid = each_obj['fid']
        latest_five = update_fanpage_data(fid)
        if latest_five is not None:
            for pid in latest_five:
                db_find = coll_post.find_one({'pid':pid})
                if db_find is None:
                    update_post_data(fid,pid)
    post_in_db = db.post.find({},{"_id":False,'pid':True})
    for p in post_in_db:
        update_post_detail_data(p['pid'])

In [68]:
test_fid_list = [{'fid':'126949930763408'},{'fid':'127581957282188'}]

In [69]:
update_db_weekly(test_fid_list)

fanpage 126949930763408 send request to FB
post 126949930763408_674545299337199 send request to FB
post 126949930763408_673905422734520 send request to FB
post 126949930763408_669660796492316 send request to FB
post 126949930763408_671004546357941 send request to FB
post 126949930763408_669573229834406 send request to FB
fanpage 127581957282188 send request to FB
post 127581957282188_1539510482755988 send request to FB
post 127581957282188_1531069253600111 send request to FB
post 127581957282188_1530654893641547 send request to FB
post 127581957282188_1524463990927304 send request to FB
post 127581957282188_1520103498030020 send request to FB
127581957282188_1531069253600111 obj has no 'shares' attribute
127581957282188_1531069253600111 cannot get 'comment_count' from Graph API
127581957282188_1531069253600111 cannot get 'friend_tagging_count' from Graph API


### [Note] Find_one_and_update method

In [121]:
from pymongo import ReturnDocument

In [132]:
coll_post.find_one_and_update({'pid':'143322382408914_1283777468363394'},
                              {'$set': { 'share_count': 444,
                                        'share':3}},
                              projection={'_id':False,'created_time':False,'fid':False,'message':False,'post_type':False,'date':False},
                              return_document=True)

{u'pid': u'143322382408914_1283777468363394', u'share': 3, u'share_count': 444}